# NAMES: Timothy Barao, Marlan McInnes-Taylor¶
# FSUIDS: tjb13b, mm05f

# Sklearn was used for part 1
# Part 2 utilizes the code for EM found here: https://people.duke.edu/~ccc14/sta-663/EMAlgorithm.html

In [1]:
from sklearn import mixture
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
x1_txt = "../data/x1.txt"
x2_txt = "../data/x2.txt"
xeasy_txt = "../data/xeasy.txt"

In [3]:
# Load in datasets
x1 = np.genfromtxt(x1_txt, delimiter=',')
x2 = np.genfromtxt(x2_txt, delimiter=',')
xeasy = np.genfromtxt(xeasy_txt, delimiter=',')

In [4]:
# Set k's to use
K = [1, 2]

# Part 1

# xeasy

In [5]:
for k in K:
    gmm = mixture.GaussianMixture(init_params = "kmeans", max_iter = 100,  n_components = k)
    gmm.fit(xeasy)

    rd = 5
    print("k =", k)
    print("pi_k: \n", np.round(gmm.weights_, rd))
    print("mu_k: \n", np.round(gmm.means_, rd))
    print("sigma_k: \n", np.round(gmm.covariances_, rd))
    print("")

k = 1
pi_k: 
 [1.]
mu_k: 
 [[1.79627 1.15065]]
sigma_k: 
 [[[ 3.17198 -2.27654]
  [-2.27654  3.49633]]]

k = 2
pi_k: 
 [0.40719 0.59281]
mu_k: 
 [[ 0.02246  3.07758]
 [ 3.01465 -0.17291]]
sigma_k: 
 [[[ 1.01104 -0.04961]
  [-0.04961  0.94407]]

 [[ 1.01065  0.15418]
  [ 0.15418  0.94718]]]



# X1

In [6]:
for k in K:
    gmm = mixture.GaussianMixture(init_params = "kmeans", max_iter = 100,  n_components = k)
    gmm.fit(x1)

    rd = 5
    print("k =", k)
    print("pi_k: \n", np.round(gmm.weights_, rd))
    print("mu_k: \n", np.round(gmm.means_, rd))
    print("sigma_k: \n", np.round(gmm.covariances_, rd))
    print("") 

k = 1
pi_k: 
 [1.]
mu_k: 
 [[0.65826 1.33115]]
sigma_k: 
 [[[ 2.49394 -0.83207]
  [-0.83207  2.57821]]]

k = 2
pi_k: 
 [0.56407 0.43593]
mu_k: 
 [[-0.27931  2.14906]
 [ 1.87145  0.2728 ]]
sigma_k: 
 [[[ 1.57701  0.34995]
  [ 0.34995  2.09335]]

 [[ 1.07115 -0.08532]
  [-0.08532  1.21986]]]



# x2

In [7]:
for k in K:
    gmm = mixture.GaussianMixture(init_params = "kmeans", max_iter = 100,  n_components = k)
    gmm.fit(x2)

    rd = 5
    print("k =", k)
    print("pi_k: \n", np.round(gmm.weights_, rd))
    print("mu_k: \n", np.round(gmm.means_, rd))
    print("sigma_k: \n", np.round(gmm.covariances_, rd))
    print("")

k = 1
pi_k: 
 [1.]
mu_k: 
 [[ 0.09421 -0.08491]]
sigma_k: 
 [[[5.06894 0.40546]
  [0.40546 5.05081]]]

k = 2
pi_k: 
 [0.4385 0.5615]
mu_k: 
 [[ 0.17023 -0.14255]
 [ 0.03484 -0.0399 ]]
sigma_k: 
 [[[ 9.86732  0.79962]
  [ 0.79962 10.26301]]

 [[ 1.31366  0.10374]
  [ 0.10374  0.97578]]]



# Part 2

# EM Code from: https://people.duke.edu/~ccc14/sta-663/EMAlgorithm.html

# This was done for ease of implementing the two-step EM

In [8]:
from scipy.stats import multivariate_normal as mvn

def em(xs, pis, mus, sigmas, tol=0.01, max_iter=1000):

    n, p = xs.shape
    k = len(pis)

    ll_old = 0
    for i in range(max_iter):
        exp_A = []
        exp_B = []
        ll_new = 0

        # E-step
        ws = np.zeros((k, n))
        for j in range(len(mus)):
            for i in range(n):
                ws[j, i] = pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
        ws /= ws.sum(0)

        # M-step
        pis = np.zeros(k)
        for j in range(len(mus)):
            for i in range(n):
                pis[j] += ws[j, i]
        pis /= n

        mus = np.zeros((k, p))
        for j in range(k):
            for i in range(n):
                mus[j] += ws[j, i] * xs[i]
            mus[j] /= ws[j, :].sum()

        sigmas = np.zeros((k, p, p))
        for j in range(k):
            for i in range(n):
                ys = np.reshape(xs[i]- mus[j], (2,1))
                sigmas[j] += ws[j, i] * np.dot(ys, ys.T)
            sigmas[j] /= ws[j,:].sum()

        # update complete log likelihoood
        ll_new = 0.0
        for i in range(n):
            s = 0
            for j in range(k):
                s += pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
            ll_new += np.log(s)

        if np.abs(ll_new - ll_old) < tol:
            break
        ll_old = ll_new

    return pis, mus, sigmas

In [29]:
def provableEM(k, data):
    # Build S in pruning step
    S = [[], [] ,[]]
    
    # 4 was one of the suggest values by the professor to use here, 3 could have worked also 
    l = 4
    
    # Initialize pi
    pi = [1/l]*l
    
    # Initialize mu with random data points
    rands = np.random.randint(0, data.shape[0]-1, l)
    mu = [data[i] for i in rands]
    
    # Initialize sigma with equation on slide 19 of LearningGM
    sigma = [np.delete(x - mu, i, 0) for i,x in enumerate(mu)]
    sigma = [min(np.linalg.norm(i, axis=1)) for i in sigma]
    sigma = np.array([x**2 * np.eye(2) for x in sigma]) 

    # First EM step
    pi, mu, sigma = em(data, pi, mu, sigma) 
    
    # Pruning
    # Remove all clusters where pi < 1/4 * l
    indcs = np.argwhere(pi < 1/(4*l))
    
    # Keep cluster only where pi >= 1/4 * l
    pi = pi[pi >= 1/(4*l)]
    mu = np.delete(mu, indcs, axis=0)
    sigma = np.delete(sigma, indcs, axis=0)
 
    # Only want to do this step if we end up with more clusters than k
    if len(pi) > k:
        # Choose random cluster to add to S, as stated in LearningGM slides
        rand_c = np.random.randint(0, len(pi) - 1, 1)[0]

        S[0].append(pi[rand_c])
        S[1].append(mu[rand_c])
        S[2].append(sigma[rand_c])
        
        # Now add to S the center with the maximum distance, but only when k = 2
        # Otherwise skip this step, since there would only be one cluster
        if k == 2:
            # Distance formula, get distance from every mu
            dist = [[np.sum((S[0][0] - mu[i])**2), i] for i in range(len(mu))]
            
            # Sort the distances to determine the max distance
            dist.sort(reverse = True, key=lambda x: x[0])
            max_d = [x[1] for x in dist[1]]
            
            # add cluster to S
            S[0] += [pi[j] for j in max_d]
            S[1] += [mu[j] for j in max_d]
            S[2] += [sigma[j] for j in max_d]

    # Return the second EM step
    return em(data, np.array(S[0]), np.array(S[1]), np.array(S[2])) 
    
    
    


# xeasy

In [30]:
for k in K:
    rd = 5
    pi, mu, sigma = provableEM(k, xeasy)
    
    print("k =", k)
    print("pi_k: \n", np.round(pi, rd))
    print("mu_k: \n", np.round(mu, rd))
    print("sigma_k: \n", np.round(sigma, rd))
    print("")

k = 1
pi_k: 
 [1.]
mu_k: 
 [[1.79627 1.15065]]
sigma_k: 
 [[[ 3.17198 -2.27654]
  [-2.27654  3.49633]]]

k = 2
pi_k: 
 [0.59187 0.40813]
mu_k: 
 [[ 3.01705 -0.17528]
 [ 0.02585  3.07355]]
sigma_k: 
 [[[ 1.00756  0.15752]
  [ 0.15752  0.94435]]

 [[ 1.01521 -0.05471]
  [-0.05471  0.9501 ]]]



# X1

In [31]:
for k in K:
    rd = 5
    pi, mu, sigma = provableEM(k, x1)
    
    print("k =", k)
    print("pi_k: \n", np.round(pi, rd))
    print("mu_k: \n", np.round(mu, rd))
    print("sigma_k: \n", np.round(sigma, rd))
    print("")

k = 1
pi_k: 
 [1.]
mu_k: 
 [[0.65826 1.33115]]
sigma_k: 
 [[[ 2.49393 -0.83207]
  [-0.83207  2.57821]]]

k = 2
pi_k: 
 [0.36859 0.63141]
mu_k: 
 [[ 2.042    0.17343]
 [-0.14949  2.00696]]
sigma_k: 
 [[[ 0.89157 -0.05856]
  [-0.05856  1.11194]]

 [[ 1.65913  0.19744]
  [ 0.19744  2.19501]]]



# X2

In [32]:
for k in K:
    rd = 5
    pi, mu, sigma = provableEM(k, x2)
    
    print("k =", k)
    print("pi_k: \n", np.round(pi, rd))
    print("mu_k: \n", np.round(mu, rd))
    print("sigma_k: \n", np.round(sigma, rd))
    print("")

k = 1
pi_k: 
 [1.]
mu_k: 
 [[ 0.09421 -0.08491]]
sigma_k: 
 [[[5.06894 0.40546]
  [0.40546 5.05081]]]

k = 2
pi_k: 
 [0.49607 0.50393]
mu_k: 
 [[ 0.17787 -0.12277]
 [ 0.01185 -0.04764]]
sigma_k: 
 [[[9.15992 0.76225]
  [0.76225 9.29089]]

 [[1.02802 0.06042]
  [0.06042 0.87399]]]

